# Step 1: Data Preparation

### Features in the Dataset

Numerical Features: Temperature, Humidity, Moisture, Nitrogen (N), Phosphorous (P), Potassium (K).
Categorical Features: Soil Type (Sandy, Loamy, Black, etc.).
Target Variable: Crop Type (e.g., Maize, Sugarcane, Cotton).
Preprocessing

### Encode Categorical Features:

Use one-hot encoding for Soil Type.

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv("crop_data.csv")
encoder = OneHotEncoder(sparse_output=False)
soil_encoded = encoder.fit_transform(data[["Soil Type"]])
soil_df = pd.DataFrame(soil_encoded, columns=encoder.get_feature_names_out(["Soil Type"]))
data = pd.concat([data.drop("Soil Type", axis=1), soil_df], axis=1)

## Handle Missing Values:

Replace zeros in Nitrogen, Phosphorous, and Potassium with domain-specific defaults or impute using the mean/median.

## Normalize Numerical Features:

In [2]:
from sklearn.preprocessing import StandardScaler

numerical_features = ["Temparature", "Humidity", "Moisture", "Nitrogen", "Potassium", "Phosphorous"]
scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Step 2: Model Building (Crop Recommendation)

### Algorithm Choice

Use **Random Forest** or **Gradient Boosting (XGBoost)** for classification. These handle mixed data types and non-linear relationships.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = data.drop(["Crop Type", "Fertilizer Name"], axis=1)
y = data["Crop Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier()

## Evaluation

In [4]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.089375
              precision    recall  f1-score   support

      Barley       0.09      0.11      0.10       144
      Cotton       0.09      0.11      0.10       141
 Ground Nuts       0.09      0.10      0.09       145
       Maize       0.08      0.10      0.09       147
     Millets       0.16      0.12      0.14       149
   Oil seeds       0.16      0.14      0.15       154
       Paddy       0.06      0.05      0.06       146
      Pulses       0.08      0.09      0.09       125
   Sugarcane       0.07      0.05      0.06       174
     Tobacco       0.06      0.07      0.07       130
       Wheat       0.05      0.05      0.05       145

    accuracy                           0.09      1600
   macro avg       0.09      0.09      0.09      1600
weighted avg       0.09      0.09      0.09      1600



# Step 3: Fertilizer Recommendation (Rule-Based)

Use NPK values to suggest fertilizers:

- **Low Nitrogen:** Recommend Urea.

- **Low Phosphorous:** Recommend DAP (Di-ammonium Phosphate).

- **Low Potassium:** Recommend MOP (Muriate of Potash).

In [5]:
def recommend_fertilizer(n, p, k):
    if n < 20:  # Example threshold
        return "Urea"
    elif p < 20:
        return "DAP"
    elif k < 20:
        return "MOP"
    else:
        return "Balanced (No fertilizer needed)"

# Step 4: Dynamic Recommendations

- **Crop Rotation:**

Use historical crop data to suggest rotations (e.g., follow wheat with lentils for nitrogen fixation).

- **Weather Integration:**

Fetch real-time temperature/rainfall data via APIs (e.g., OpenWeatherMap) to adjust recommendations.

## Step 5: Deployment (Dashboard)

Use tools like Streamlit or Dash to build a farmer-friendly interface:

In [6]:
import streamlit as st

st.title("Crop Recommendation System")
temp = st.number_input("Temperature (°C)")
humidity = st.number_input("Humidity (%)")
moisture = st.number_input("Moisture (%)")
soil_type = st.selectbox("Soil Type", ["Sandy", "Loamy", "Black", "Red", "Clayey"])

if st.button("Recommend"):
    input_data = preprocess_input(temp, humidity, moisture, soil_type)  # Preprocess like Step 1
    crop = model.predict(input_data)
    fertilizer = recommend_fertilizer(input_data["Nitrogen"], input_data["Phosphorous"], input_data["Potassium"])
    st.write(f"**Recommended Crop**: {crop}")
    st.write(f"**Fertilizer**: {fertilizer}")

2025-02-26 07:37:40.166 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 07:37:40.379 
  command:

    streamlit run /home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-26 07:37:40.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 07:37:40.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 07:37:40.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 07:37:40.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 07:37:40.382 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 07:37:40.382 Session state 